
# Unit 2 — Team Classification (Flights, BQML)

**Goal (team):** Build an *ops-ready* classifier in **BigQuery ML** to predict **`diverted`** on U.S. flights. Minimal handholding by design.

**What you deliver (inside this notebook):**
- One **LOGISTIC_REG** model (baseline), one **engineered** model using `TRANSFORM`
- **Evaluation** via `ML.EVALUATE` and **confusion matrices** (default 0.5 + your custom threshold)
- **Threshold choice** + 3–5 sentence ops justification
- Embedded **rubric** below (self-check before submission)

> Choose *one* dataset table that exists at your institution:  
> • `bigquery-public-data.faa.us_flights` **or** `bigquery-public-data.flights.*`  
> Make sure the table has `carrier`, `dep_delay`, `arr_delay` (for filters), `origin`, `dest`, `diverted` (or equivalent).


In [1]:

# --- Minimal setup (edit 3 vars) ---
from google.colab import auth
auth.authenticate_user()

import os
from google.cloud import bigquery

PROJECT_ID = "upbeat-aspect-471118-v8"      # e.g., mgmt-467-47888
REGION     = "US"
TABLE_PATH = "upbeat-aspect-471118-v8.flights.Flights"
DATASET_ID = TABLE_PATH.split('.')[1]

os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["REGION"]     = REGION
bq = bigquery.Client(project=PROJECT_ID)

print("BQ Project:", PROJECT_ID)
print("Source table:", TABLE_PATH)


BQ Project: upbeat-aspect-471118-v8
Source table: upbeat-aspect-471118-v8.flights.Flights


### Quick sanity check

In [2]:

preview_sql = f"SELECT * FROM `{TABLE_PATH}` LIMIT 5"
bq.query(preview_sql).result().to_dataframe()


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1999,3,9,10,5,1999-09-10,DL,19790,DL,N241DL,...,,,,None,None,,None,None,,
1,1999,3,8,25,3,1999-08-25,DL,19790,DL,N222DZ,...,,,,None,None,,None,None,,
2,1999,1,2,9,2,1999-02-09,DL,19790,DL,N959DL,...,,,,None,None,,None,None,,
3,1989,1,2,1,3,1989-02-01,EA,19707,EA,,...,,,,None,None,,None,None,,
4,1997,4,12,16,2,1997-12-16,DL,19790,DL,N374DL,...,,,,None,None,,None,None,,



## 1) Canonical mapping (adjust as needed)
Map to a minimal schema used in the rest of the notebook:
- `flight_date` (DATE), `dep_delay` (NUM), `distance` (NUM), `carrier` (STRING), `origin` (STRING), `dest` (STRING), `diverted` (BOOL)


In [3]:
# Adjust ONLY if your table uses different column names.
CANONICAL_BASE_SQL = f'''
WITH canonical_flights AS (
  SELECT
    CAST(FlightDate AS DATE) AS flight_date,
    CAST(DepDelay AS FLOAT64) AS DepDelay,
    CAST(Distance  AS FLOAT64) AS distance,
    CAST(Reporting_Airline   AS STRING)  AS carrier,
    CAST(Origin    AS STRING)  AS origin,
    CAST(COALESCE(dest, Dest) AS STRING) AS dest,
    CAST((CASE WHEN SAFE_CAST(Diverted AS INT64)=1 OR LOWER(CAST(Diverted AS STRING))='true' THEN TRUE ELSE FALSE END) AS BOOL) AS diverted
  FROM `{TABLE_PATH}`
  WHERE DepDelay IS NOT NULL
)
'''
print(CANONICAL_BASE_SQL[:600] + "\n...")



WITH canonical_flights AS (
  SELECT
    CAST(FlightDate AS DATE) AS flight_date,
    CAST(DepDelay AS FLOAT64) AS DepDelay,
    CAST(Distance  AS FLOAT64) AS distance,
    CAST(Reporting_Airline   AS STRING)  AS carrier,
    CAST(Origin    AS STRING)  AS origin,
    CAST(COALESCE(dest, Dest) AS STRING) AS dest,
    CAST((CASE WHEN SAFE_CAST(Diverted AS INT64)=1 OR LOWER(CAST(Diverted AS STRING))='true' THEN TRUE ELSE FALSE END) AS BOOL) AS diverted
  FROM `upbeat-aspect-471118-v8.flights.Flights`
  WHERE DepDelay IS NOT NULL
)

...


### 2) Split (80/20)

In [4]:
SPLIT_CLAUSE = r'''
, split AS (
  SELECT cf.*,
         CASE WHEN RAND() < 0.8 THEN 'TRAIN' ELSE 'EVAL' END AS data_split
  FROM canonical_flights cf
)
'''
print(SPLIT_CLAUSE)



, split AS (
  SELECT cf.*,
         CASE WHEN RAND() < 0.8 THEN 'TRAIN' ELSE 'EVAL' END AS data_split
  FROM canonical_flights cf
)



\
## 3) Baseline model - Model B

Use **only** a small set of signals for the baseline (keep it honest).


In [18]:
# Create the dataset if it doesn't exist
# Create the dataset if it doesn't exist
dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset.location = REGION

try:
    bq.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {PROJECT_ID}.{DATASET_ID} created or already exists.")
except Exception as e:
    print(f"Error creating dataset {PROJECT_ID}.{DATASET_ID}: {e}")


MODEL_B_ENGINEERED = f"{PROJECT_ID}.{DATASET_ID}.model_b_engineered"

sql_create_model_b = f'''
CREATE OR REPLACE MODEL `{MODEL_B_ENGINEERED}`
OPTIONS (
  MODEL_TYPE='LOGISTIC_REG',
  INPUT_LABEL_COLS=['diverted']
) AS
{CANONICAL_BASE_SQL.strip()}
{SPLIT_CLAUSE.strip()}
SELECT
  diverted,
  DepDelay,
  distance,
  carrier,
  origin,
  dest,
  CONCAT(origin, '-', dest) AS route,
  EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week,
  EXTRACT(MONTH FROM flight_date)     AS month,
  CASE
    WHEN DepDelay < 0 THEN 'early'
    WHEN DepDelay = 0 THEN 'on_time'
    WHEN DepDelay > 0 AND DepDelay <= 30 THEN 'minor'
    WHEN DepDelay > 30 AND DepDelay <= 120 THEN 'moderate'
    WHEN DepDelay > 120 THEN 'major'
    ELSE 'unknown'
  END AS dep_delay_bucket
FROM split
WHERE data_split='TRAIN'
;
'''

print("Training Model B (Engineered)....")
job_model_b = bq.query(sql_create_model_b)
job_model_b.result()
print("Model B trained:", MODEL_B_ENGINEERED)


Dataset upbeat-aspect-471118-v8.flights created or already exists.
Training Model B (Engineered)....
Model B trained: upbeat-aspect-471118-v8.flights.model_b_engineered


Evaluate Model B

In [25]:
MODEL_B_ENGINEERED = f"{PROJECT_ID}.{DATASET_ID}.model_b_engineered"
sql_eval_b = f"""
SELECT *
FROM ML.EVALUATE(
  MODEL `{MODEL_B_ENGINEERED}`,
  (
    {CANONICAL_BASE_SQL.strip()}
    {SPLIT_CLAUSE.strip()}
    SELECT
      diverted,
      depdelay,
      distance,
      carrier,
      origin,
      dest,
      CONCAT(origin, '-', dest) AS route,
      EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week,
      EXTRACT(MONTH FROM flight_date)     AS month,
      CASE
        WHEN depdelay < 0 THEN 'early'
        WHEN depdelay = 0 THEN 'on_time'
        WHEN depdelay > 0 AND depdelay <= 30 THEN 'minor'
        WHEN depdelay > 30 AND depdelay <= 120 THEN 'moderate'
        WHEN depdelay > 120 THEN 'major'
        ELSE 'unknown'
      END AS dep_delay_bucket
    FROM split
    WHERE data_split = 'EVAL'
  )
);
"""

eval_b = bq.query(sql_eval_b).result().to_dataframe()
eval_b

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.8,0.004348,0.997665,0.008649,0.015119,0.802097


### Confusion matrix — your custom threshold - 0.05

In [7]:

CUSTOM_THRESHOLD = 0.05  # TODO: justify in ops terms

cm_thresh_sql = f'''
{CANONICAL_BASE_SQL.strip()}
{SPLIT_CLAUSE.strip()}

, scored AS (
  SELECT
    t.diverted AS label,
    CAST(t.predicted_diverted_probs[OFFSET(0)].prob >= {CUSTOM_THRESHOLD} AS BOOL) AS pred_label
  FROM ML.PREDICT(MODEL `{MODEL_B_ENGINEERED}`,
      (SELECT
         diverted,
         DepDelay,
         distance,
         carrier,
         origin,
         dest,
         CONCAT(origin, '-', dest) AS route,
         EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week,
         EXTRACT(MONTH FROM flight_date)     AS month,
         CASE
           WHEN DepDelay < 0 THEN 'early'
           WHEN DepDelay = 0 THEN 'on_time'
           WHEN DepDelay > 0 AND DepDelay <= 30 THEN 'minor'
           WHEN DepDelay > 30 AND DepDelay <= 120 THEN 'moderate'
           WHEN DepDelay > 120 THEN 'major'
           ELSE 'unknown'
         END AS dep_delay_bucket
       FROM split
       WHERE data_split='EVAL')
  ) AS t
)
SELECT
  SUM(CASE WHEN label=TRUE  AND pred_label=TRUE  THEN 1 ELSE 0 END) AS TP,
  SUM(CASE WHEN label=FALSE AND pred_label=TRUE  THEN 1 ELSE 0 END) AS FP,
  SUM(CASE WHEN label=TRUE  AND pred_label=FALSE THEN 1 ELSE 0 END) AS FN,
  SUM(CASE WHEN label=FALSE AND pred_label=FALSE THEN 1 ELSE 0 END) AS TN
FROM scored;
'''
bq.query(cm_thresh_sql).result().to_dataframe()


,TP,FP,FN,TN
0,31,377,863,391441


### Confusion matrix — default 0.5 threshold

In [8]:

cm_default_sql = f'''
{CANONICAL_BASE_SQL.strip()}
{SPLIT_CLAUSE.strip()}
, scored AS (
  SELECT
    t.diverted AS label,
    t.predicted_diverted AS pred_label,
    t.predicted_diverted_probs[OFFSET(0)].prob AS score
  FROM ML.PREDICT(MODEL `{MODEL_B_ENGINEERED}`,
      (SELECT
         diverted,
         DepDelay,
         distance,
         carrier,
         origin,
         dest,
         CONCAT(origin, '-', dest) AS route,
         EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week,
         EXTRACT(MONTH FROM flight_date)     AS month,
         CASE
           WHEN DepDelay < 0 THEN 'early'
           WHEN DepDelay = 0 THEN 'on_time'
           WHEN DepDelay > 0 AND DepDelay <= 30 THEN 'minor'
           WHEN DepDelay > 30 AND DepDelay <= 120 THEN 'moderate'
           WHEN DepDelay > 120 THEN 'major'
           ELSE 'unknown'
         END AS dep_delay_bucket
       FROM split
       WHERE data_split='EVAL')
  ) AS t
)
SELECT
  SUM(CASE WHEN label=TRUE  AND pred_label=TRUE  THEN 1 ELSE 0 END) AS TP,
  SUM(CASE WHEN label=FALSE AND pred_label=TRUE  THEN 1 ELSE 0 END) AS FP,
  SUM(CASE WHEN label=TRUE  AND pred_label=FALSE THEN 1 ELSE 0 END) AS FN,
  SUM(CASE WHEN label=FALSE AND pred_label=FALSE THEN 1 ELSE 0 END) AS TN
FROM scored;
'''
bq.query(cm_default_sql).result().to_dataframe()


,TP,FP,FN,TN
0,5,4,922,391375



## 4) Engineered model D
Goal: choose an operating threshold that minimizes expected cost.


In [13]:
# --------------------------------------
# MODEL D — THRESHOLD & COST POLICY
# --------------------------------------

MODEL_D_RESULTS = f"{PROJECT_ID}.{DATASET_ID}.model_d_threshold_cost"

# Define cost matrix
C_FP = 1000
C_FN = 6000

# Generate scored predictions on the EVAL split
sql_model_d = f'''
CREATE OR REPLACE TABLE `{MODEL_D_RESULTS}` AS
SELECT
  *,
  -- Extract probability of diverted = TRUE
  (
    SELECT prob
    FROM UNNEST(predicted_diverted_probs)
   WHERE Cast(label AS STRING) = 'true') AS prob_diverted,
  diverted AS actual_label
FROM
  ML.PREDICT(MODEL `{MODEL_B_ENGINEERED}`,
    (
      {CANONICAL_BASE_SQL.strip()}
      {SPLIT_CLAUSE.strip()}
      SELECT
        diverted,
        DepDelay,
        distance,
        carrier,
        origin,
        dest,
        CONCAT(origin, '-', dest) AS route,
        EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week,
        EXTRACT(MONTH FROM flight_date)     AS month,
        CASE
          WHEN DepDelay < 0 THEN 'early'
          WHEN DepDelay = 0 THEN 'on_time'
          WHEN DepDelay > 0 AND DepDelay <= 30 THEN 'minor'
          WHEN DepDelay > 30 AND DepDelay <= 120 THEN 'moderate'
          WHEN DepDelay > 120 THEN 'major'
          ELSE 'unknown'
        END AS dep_delay_bucket
      FROM split
      WHERE data_split='EVAL'
    )
  )
;
'''

print("Creating scored evaluation table for Model D...")
bq.query(sql_model_d).result()
print(f"Table created: {MODEL_D_RESULTS}")

# Compute expected cost across thresholds
sql_threshold_cost = f'''
WITH threshold_eval AS (
  SELECT
    threshold,
    SUM(CASE WHEN prob_diverted >= threshold AND actual_label = FALSE THEN {C_FP} ELSE 0 END) AS false_positive_cost,
    SUM(CASE WHEN prob_diverted <  threshold AND actual_label = TRUE  THEN {C_FN} ELSE 0 END) AS false_negative_cost,
    SUM(CASE WHEN prob_diverted >= threshold AND actual_label = FALSE THEN {C_FP}
             WHEN prob_diverted <  threshold AND actual_label = TRUE  THEN {C_FN}
             ELSE 0 END) AS total_cost,
    COUNT(*) AS total_records,
    AVG(CASE WHEN prob_diverted >= threshold AND actual_label = TRUE THEN 1 ELSE 0 END) AS recall,
    AVG(CASE WHEN prob_diverted <  threshold AND actual_label = FALSE THEN 1 ELSE 0 END) AS specificity
  FROM `{MODEL_D_RESULTS}`,
  UNNEST(GENERATE_ARRAY(0.0, 1.0, 0.01)) AS threshold
  GROUP BY threshold
)
SELECT * FROM threshold_eval
ORDER BY total_cost ASC
LIMIT 1;
'''

print("Finding optimal threshold based on cost minimization...")
best_threshold = bq.query(sql_threshold_cost).result().to_dataframe()
print("Optimal threshold and expected cost:")
print(best_threshold)


Creating scored evaluation table for Model D...
Table created: upbeat-aspect-471118-v8.flights.model_d_threshold_cost
Finding optimal threshold based on cost minimization...
Optimal threshold and expected cost:
   threshold  false_positive_cost  false_negative_cost  total_cost  \
0       0.92                    0              5358000     5358000   

   total_records    recall  specificity  
0         393201  0.000015     0.997714  



### ✈️ Model D — Threshold & Cost Policy Explanation

The optimal threshold of **0.92** for Model D was selected to minimize total operational costs for the airline by balancing the financial impact of **False Positives (FP)** and **False Negatives (FN)**.  

A **False Positive** — predicting a diversion that never occurs — costs **$1,000** due to unnecessary actions such as rerouting planes, preparing ground crews, or holding gates, leading to wasted resources and operational disruptions.  

In contrast, a **False Negative** — failing to predict a diversion that actually happens — costs **$6,000**, as the airline must reactively manage the event, resulting in passenger delays, last-minute gate changes, and potential penalties or customer dissatisfaction.  

By setting a **high threshold (0.92)**, the model operates conservatively, nearly eliminating false positives (reflected in a **$0 FP cost**).  
Although this increases the number of false negatives, the total expected cost remains lowest.  
This threshold therefore prioritizes avoiding costly false alarms over capturing every diversion, aligning best with the airline’s overall operational cost strategy.




---

## Rubric (Flights, 100 pts)
**Team-only deliverable in this notebook**

- Baseline LOGISTIC_REG + evaluation (AUC + confusion @0.5) — **20**  
- Custom threshold confusion matrix + ops justification — **20**  
- Engineered model with `TRANSFORM` (route, DOW, delay bucket) — **20**  
- Comparison table (baseline vs engineered) + 3–5 sentence interpretation — **20**  
- Reproducibility: parameters clear, no hidden magic; schema mapping documented — **10**  
- Governance notes: assumptions/limitations + slices you would monitor — **10**

> **Strictness:** No screenshots; use actual results cells. Keep explanations concise (bullet points OK).
